In [1]:
from llama_index.llms.groq import Groq
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.vector_stores.deeplake import DeepLakeVectorStore
from llama_index.core.storage.storage_context import StorageContext
from llama_index.core import VectorStoreIndex
import os

from dotenv import load_dotenv

load_dotenv()

/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/deeplake/util/check_latest_version.py:32: UserWarning: A newer version of deeplake (3.9.8) is available. It's recommended that you update to the latest version using `pip install -U deeplake`.
  warnings.warn(


True

In [2]:
## embedding model
def get_embedding_model(model_name="/teamspace/studios/this_studio/bge-small-en-v1.5"):
    embed_model = HuggingFaceEmbedding(model_name=model_name)
    return embed_model

# generator model
def get_llm(model_name="llama3-8b-8192"):
    llm = Groq(model=model_name, api_key=os.getenv("GROQ_API"), temperature=0.8)
    return llm

## get deeplake vector database
def get_vector_database(id, dataset_name):
    my_activeloop_org_id = id # "hunter"
    my_activeloop_dataset_name = dataset_name # "Vietnamese-law-RAG"
    dataset_path = f"hub://{my_activeloop_org_id}/{my_activeloop_dataset_name}"
    vector_store = DeepLakeVectorStore(
        dataset_path=dataset_path,
        overwrite=False,
    )
    return vector_store

def get_index(vector_store):
    index = VectorStoreIndex.from_vector_store(vector_store=vector_store)
    return index

In [4]:
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core import Settings
from llama_index.llms.ollama import Ollama

## set embedding model
embed_model = get_embedding_model()
Settings.embed_model = embed_model

## set llm 
llm = Ollama(model="vistral", request_timeout=120.0, max_new_tokens=2000) # get_llm()
Settings.llm = llm

## get vector store, index, and two retrievers
vector_store = get_vector_database("hunter", "Vietnamese-legal-data")
index = get_index(vector_store=vector_store)
vector_retriever = index.as_retriever(similarity_top_k=3)

Deep Lake Dataset in hub://hunter/Vietnamese-legal-data already exists, loading from the storage


In [6]:
from llama_index.core.indices.query.query_transform import HyDEQueryTransform
from llama_index.core.query_engine import TransformQueryEngine
from IPython.display import Markdown, display

In [8]:
query_str = "Vượt đèn đỏ thì bị phạt bao nhiêu tiền?"
query_engine = index.as_query_engine()
response = query_engine.query(query_str)
display(Markdown(f"<b>{response}</b>"))

<b>
Bạn có thể tham khảo Nghị định 100/2019/NĐ-CP để biết mức xử lý đối với hành vi vượt đèn đỏ. Theo đó, người điều khiển xe máy vượt đèn đỏ sẽ bị phạt từ 600.000 đồng đến 1.000.000 đồng và giữ bằng lái từ 1 tháng tới 3 tháng. Nếu gây tai nạn giao thông thì có thể bị cấm hành nghề lên tới 4 tháng. Mức xử lý này áp dụng với xe máy, còn ô tô vượt đèn đỏ sẽ bị phạt tiền từ 3 triệu đến 5 triệu đồng, tạm giữ giấy phép lái xe từ 01 tháng đến 03 tháng và tước quyền sử dụng Giấy phép lái xe từ 02 tháng đến 04 tháng. Nếu gây tai nạn giao thông thì có thể bị cấm hành nghề lên tới 6 tháng (theo Khoản 5 Điều 5 Nghị định 100/2019/NĐ-CP). </b>

In [14]:
llm = Ollama(model="vistral", request_timeout=120.0, max_new_tokens=2000) # get_llm()
Settings.llm = llm

In [15]:
hyde = HyDEQueryTransform(include_original=True)
hyde_query_engine = TransformQueryEngine(query_engine, hyde)
response = hyde_query_engine.query(query_str)
display(Markdown(f"<b>{response}</b>"))

<b>
Mức phạt đối với hành vi điều khiển xe máy vượt đèn đỏ sẽ là từ 600 - 1 triệu đồng (điểm a khoản 5 Điều 6 Nghị định 100/2019/NĐ-CP). Ngoài ra, người vi phạm còn bị tước quyền sử dụng Giấy phép lái xe từ 01 tháng đến 03 tháng.

Ngoài những trường hợp nêu trên thì một số lỗi khác cũng phải chịu mức phạt nặng như: vượt đèn vàng có thể bị phạt tiền từ 4 - 6 triệu đồng (điểm b khoản 5 Điều 5 Nghị định 100/2019/NĐ-CP). Với hành vi dừng xe, đỗ xe gây cản trở giao thông tại khu vực nhà ga, bến xe, sân bay; vượt đèn vàng khi tham gia giao thông có thể bị phạt tiền từ 4 - 6 triệu đồng. Điều khiển xe ô tô không có đủ đèn chiếu sáng khi trời tối hoặc khi sương mù, thời tiết xấu làm hạn chế tầm nhìn cũng sẽ bị xử phạt nặng (điểm b khoản 2 Điều 5 Nghị định số 100/2019/NĐ-CP). Người điều khiển phương tiện sử dụng điện thoại di động khi đang lái xe ô tô chạy trên đường có thể bị phạt tiền từ 600.000 đồng đến 1 triệu đồng (điểm l khoản 3 Điều 5 Nghị định số 100/2019/NĐ-CP). Khi điều khiển phương tiện, người tham gia giao thông cũng không được phép sử dụng ô để che nắng, che mưa hoặc các vật khác làm cản trở tầm nhìn của người lái xe. Nếu vi phạm sẽ bị phạt tiền từ 600 - 800 nghìn đồng (điểm e khoản 2 Điều 5 Nghị định số 100/2019/NĐ-CP). Khi tham gia giao thông, nếu không chấp hành hiệu lệnh dừng xe của người thi hành công vụ thì mức xử phạt có thể lên đến từ 3 triệu đồng đến 4 triệu đồng. Nếu điều khiển phương tiện quá tốc độ quy định trên 20 km/h sẽ bị phạt tiền từ 5 - 7 triệu đồng (điểm b khoản 6 Điều 5 Nghị định số 100/2019/NĐ-CP). Đối với hành vi không chấp hành hiệu lệnh dừng xe của người thi hành công vụ có thể bị xử phạt đến mức tối đa là 40 triệu đồng. Nếu điều khiển phương tiện chạy quá tốc độ quy định trên 35 km/h sẽ bị phạt tiền từ 8 - 12 triệu đồng (điểm b khoản 7 Điều 5 Nghị định số 100/2019/NĐ-CP). Khi tham gia giao thông, người dân tuyệt đối không được phép sử dụng phương tiện để cản trở hoặc làm hư hỏng các công trình đường sắt. Nếu vi phạm có thể bị phạt tiền từ 6 - 8 triệu đồng (điểm a khoản 3 Điều 5 Nghị định số 100/2019/NĐ-CP). Trường hợp điều khiển xe mô tô, gắn máy không đội mũ bảo hiểm hoặc đội mà cài quai không đúng quy cách sẽ bị xử phạt đến mức tối đa là 400 nghìn đồng (điểm c khoản 3 Điều 6 Nghị định số 100/2019/NĐ-CP). Nếu người điều khiển xe ô tô khi có giấy phép lái xe nhưng đã hết hạn sử dụng dưới 6 tháng thì sẽ bị xử phạt từ 4 - 6 triệu đồng. Mức cao nhất của mức phạt này là đối với hành vi không chấp hành hiệu lệnh dừng xe hoặc tốc độ của người thi hành công vụ (điểm b khoản 7 Điều 5 Nghị định số 100/2019/NĐ-CP). Đối với những lỗi nêu trên, ngoài việc bị xử phạt tiền còn có thể áp dụng hình thức xử lý bổ sung như: tịch thu tang vật, phương tiện; tước quyền sử dụng giấy phép lái xe từ 01 tháng đến 03 tháng. Trong trường hợp cần thiết, người vi phạm sẽ phải thi lại giấy phép lái xe. </b>